In [ ]:
! pip install transformers[sentencepiece] -Uqq

     |████████████████████████████████| 4.0 MB 21.2 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 52.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.3 MB/s 
     |████████████████████████████████| 596 kB 33.5 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 


In [ ]:
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding

In [ ]:
import pandas as pd

*Enter the dataset file name*

In [ ]:
file_name = '<file_name>.csv'

In [ ]:
dataset = pd.read_csv(file_name)

In [ ]:
import matplotlib.pyplot as plt
dataset['labels'].hist()

In [ ]:
dataset.head()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
def preprocess(dataset,data_column):
  """Performs data preprocessing on the given dataset."""
  dataset[f"{data_column}_processed"] = dataset[data_column].apply(lambda x: " ".join(x.lower() for x in x.split()))
  dataset[f"{data_column}_processed"]  = dataset[f"{data_column}_processed"].str.replace('[^\w\s]','')
  return dataset

In [ ]:
dataset = preprocess(dataset, 'feedback')

In [ ]:
dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dataset['labels'] = label_encoder.fit_transform(dataset['labels'])

In [ ]:
dataset

In [ ]:
import pyarrow as pa
s = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in s.split(dataset, dataset['labels']):
  train_set = dataset.loc[train_index]
  test_set = dataset.loc[test_index]

In [ ]:
train_set.hist()
test_set.hist()

In [2]:
!pip install folium==0.2.1 -Uqq

     |████████████████████████████████| 69 kB 3.3 MB/s 


In [3]:
! pip install datasets -Uqq

     |████████████████████████████████| 325 kB 4.4 MB/s 
     |████████████████████████████████| 1.1 MB 9.4 MB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
     |████████████████████████████████| 212 kB 23.2 MB/s 
     |████████████████████████████████| 136 kB 22.3 MB/s 
     |████████████████████████████████| 127 kB 30.8 MB/s 
     |████████████████████████████████| 144 kB 27.4 MB/s 
     |████████████████████████████████| 271 kB 8.0 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 


In [ ]:
import datasets
d = datasets.DatasetDict({"train":datasets.Dataset(pa.Table.from_pandas(train_set)),"test":datasets.Dataset(pa.Table.from_pandas(test_set))})


In [ ]:
d.reset_format()

In [ ]:
checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["feedback_processed"], truncation=True)


tokenized_datasets = d.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['feedback', 'labels', 'feedback_processed', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 2989
    })
    test: Dataset({
        features: ['feedback', 'labels', 'feedback_processed', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 748
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["feedback", "__index_level_0__", "feedback_processed"])
tokenized_datasets.set_format("torch")

In [ ]:
tokenized_datasets['train'][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([  101,  6135,  2092,  2366, 16846,  2483, 10451,   102]),
 'labels': tensor(2)}

In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score
from datasets import load_metric

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test_trainer",
                                  evaluation_strategy="steps",
                                  eval_steps=1000,
                                  metric_for_best_model="accuracy",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  per_gpu_eval_batch_size=16,
                                  per_gpu_train_batch_size=16,
                                  eval_accumulation_steps=1,
                                  optim="adamw_torch"
                                  )

In [ ]:
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
trainer = Trainer(
                  args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=full_train_dataset,
                  eval_dataset=full_eval_dataset,
                  model_init=model_init,
                  tokenizer=tokenizer,
                  data_collator=data_collator
                  )

*Optuna is a software framework for automating the optimization process of these hyperparameters. It automatically finds optimal hyperparameter values by making use of different samplers such as grid search, random, bayesian, and evolutionary algorithms.*

In [ ]:
!pip install optuna -Uqq

     |████████████████████████████████| 308 kB 23.2 MB/s 
     |████████████████████████████████| 81 kB 9.7 MB/s 
     |████████████████████████████████| 210 kB 52.5 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 146 kB 62.1 MB/s 
     |████████████████████████████████| 113 kB 61.8 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 


*By default hyperparameter_search will use optuna as the backend*

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

*below code will show the best parameters*

In [ ]:
best_run.hyperparameters